# Pytorch bug

In [1]:
import torch

In [2]:
torch.set_printoptions(precision=9)

In [3]:
torch.manual_seed(1)
data = torch.autograd.Variable( torch.FloatTensor(3, 3, 4).normal_(0, 1) )[None]

In [4]:
data

tensor([[[[-1.525595903, -0.750231802, -0.653980911, -1.609484792],
          [-0.100167178, -0.609188914, -0.979772270, -1.609096289],
          [-0.712144613, 0.303721994, -0.777314305, -0.251455247]],

         [[-0.222270489, 1.687113404, 0.228425175, 0.467635512],
          [-0.696972430, -1.160761476, 0.699542403, 0.199081630],
          [0.199056506, 0.045702778, 0.152956918, -0.475678802]],

         [[-1.882142544, -0.776545048, 2.024202108, -0.086541198],
          [2.357110977, -1.037338734, 1.574798107, -0.629847229],
          [2.406978130, 0.278566241, 0.246752918, 1.184326649]]]])

### Manual executions of convolutions 

In [5]:
# First conv2d
torch.sum(data[:,:,0:3,0:3] * 0.04)

tensor(1.00000e-02 *
       1.282003708)

In [6]:
# Second conv2d
torch.sum(data[:,:,0:3,1:4] * 0.04)

tensor(1.00000e-02 *
       -9.257644415)

### Execution of convolutions via layer

In [7]:
layer = torch.nn.Conv2d(3, 1, kernel_size=3)
layer.weight.data.fill_(0.04)
layer.bias.data.fill_(0)
layer

Conv2d(3, 1, kernel_size=(3, 3), stride=(1, 1))

In [8]:
layer(data)

tensor(1.00000e-02 *
       [[[[1.282003429, -9.257646650]]]])

**--> Only the second convolution seems to be correct**

### Behaviour with multiple filters:

In [9]:
layer = torch.nn.Conv2d(3, 2, kernel_size=3)
layer.weight.data.fill_(0.04)
layer.bias.data.fill_(0)
layer

Conv2d(3, 2, kernel_size=(3, 3), stride=(1, 1))

In [10]:
layer(data)

tensor(1.00000e-02 *
       [[[[1.282002497, -9.257643670]],

         [[1.282002497, -9.257643670]]]])

**--> This result is different from the first, only by adding a filter**

Another behaviour: execute the layer on the first part of the data

In [11]:
layer(data[:,:,0:3,0:3])

tensor(1.00000e-02 *
       [[[[1.282002777]],

         [[1.282002777]]]])

**--> If I understand it correctly, this should be the same as the first column of cell 10**